In [1]:
import pandas as pd
import ast
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movies = pd.read_csv('Movies 10k.csv')
credit = pd.read_csv('Credit 10k.csv')
keywords = pd.read_csv('keywords.csv')
image = pd.read_csv('image_url.csv')

In [3]:
movies.head(2)

,adult,backdrop_path,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,genre
0,False,/zo8CIjJ2nfNOevqNajwMRO6Hwka.jpg,1241982,en,Moana 2,After receiving an unexpected call from her wa...,3664.907,/aLVkiINlIeCkcZIzb7XHzPYgO6L.jpg,2024-11-21,Moana 2,False,7.200,1555,"['Animation', 'Adventure', 'Family', 'Comedy']"
1,False,/xljLe4TiQL1b4sT7956IGgj2vrf.jpg,927342,ta,அமரன்,A heroic true story of Major Mukund Varadaraja...,2763.388,/6m435uh40N7Gzfbd69ttp6W0sdR.jpg,2024-10-31,Amaran,False,8.113,111,"['Action', 'Drama', 'Adventure', 'War']"


In [4]:
movies= movies[['id','title','overview','genre']]

In [5]:
df = movies.merge(credit, on = 'id')
df = df.merge(keywords, on = 'id')
df = df.merge(image, on = 'id')

In [6]:
df.head(2)

,id,title,overview,genre,cast,crew,keywords,image_url
0,1241982,Moana 2,After receiving an unexpected call from her wa...,"['Animation', 'Adventure', 'Family', 'Comedy']","[{'adult': False, 'gender': 1, 'id': 1564846, ...","[{'adult': False, 'gender': 2, 'id': 4524084, ...","[{'id': 658, 'name': 'sea'}, {'id': 270, 'name...",http://image.tmdb.org/t/p/w500/aLVkiINlIeCkcZI...
1,927342,Amaran,A heroic true story of Major Mukund Varadaraja...,"['Action', 'Drama', 'Adventure', 'War']","[{'adult': False, 'gender': 2, 'id': 587982, '...","[{'adult': False, 'gender': 2, 'id': 2992426, ...","[{'id': 818, 'name': 'based on novel or book'}...",http://image.tmdb.org/t/p/w500/6m435uh40N7Gzfb...


In [7]:
df.isnull().sum()

id           0
title        0
overview     2
genre        0
cast         0
crew         0
keywords     0
image_url    0
dtype: int64

In [8]:
df.dropna(inplace = True)
df.duplicated().sum()

5894

In [9]:
df.drop_duplicates(inplace = True, keep = 'first')

In [10]:
df.shape

(8911, 8)

In [11]:
df.head(2)

,id,title,overview,genre,cast,crew,keywords,image_url
0,1241982,Moana 2,After receiving an unexpected call from her wa...,"['Animation', 'Adventure', 'Family', 'Comedy']","[{'adult': False, 'gender': 1, 'id': 1564846, ...","[{'adult': False, 'gender': 2, 'id': 4524084, ...","[{'id': 658, 'name': 'sea'}, {'id': 270, 'name...",http://image.tmdb.org/t/p/w500/aLVkiINlIeCkcZI...
1,927342,Amaran,A heroic true story of Major Mukund Varadaraja...,"['Action', 'Drama', 'Adventure', 'War']","[{'adult': False, 'gender': 2, 'id': 587982, '...","[{'adult': False, 'gender': 2, 'id': 2992426, ...","[{'id': 818, 'name': 'based on novel or book'}...",http://image.tmdb.org/t/p/w500/6m435uh40N7Gzfb...


In [12]:
import ast
df['genre'] = df['genre'].apply(ast.literal_eval)

In [13]:
def cast_name(obj):
    name = []
    count = 0
    for i in ast.literal_eval(obj):
        name.append(i['name'])
        count +=1
        if count ==5:
            break
    return name

df['cast'] = df['cast'].apply(cast_name)

In [14]:
def director_name(obj):
    name = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            name.append(i['name'])
    return name

df['crew'] = df['crew'].apply(director_name)

In [15]:
def keyword_name(obj):
    name = []
    for i in ast.literal_eval(obj):
        name.append(i['name'])
    return name
df['keywords'] = df['keywords'].apply(keyword_name)

In [16]:
df.head()

,id,title,overview,genre,cast,crew,keywords,image_url
0,1241982,Moana 2,After receiving an unexpected call from her wa...,"[Animation, Adventure, Family, Comedy]","[Auliʻi Cravalho, Dwayne Johnson, Hualālai Chu...","[David G. Derrick Jr., Jason Hand, Dana Ledoux...","[sea, ocean, villain, musical, sequel, duringc...",http://image.tmdb.org/t/p/w500/aLVkiINlIeCkcZI...
1,927342,Amaran,A heroic true story of Major Mukund Varadaraja...,"[Action, Drama, Adventure, War]","[Sivakarthikeyan, Sai Pallavi, Rahul Bose, Bhu...",[Rajkumar Periasamy],"[based on novel or book, affectation, based on...",http://image.tmdb.org/t/p/w500/6m435uh40N7Gzfb...
2,822119,Captain America: Brave New World,After meeting with newly elected U.S. Presiden...,"[Action, Thriller, Science Fiction]","[Anthony Mackie, Harrison Ford, Danny Ramirez,...",[Julius Onah],"[hero, superhero, revenge, blunt, aftercredits...",http://image.tmdb.org/t/p/w500/pzIddUEMWhWzfvL...
3,939243,Sonic the Hedgehog 3,"Sonic, Knuckles, and Tails reunite against a p...","[Action, Science Fiction, Comedy, Family]","[Jim Carrey, Ben Schwartz, Keanu Reeves, Idris...",[Jeff Fowler],"[moon, sequel, based on video game, burlesque,...",http://image.tmdb.org/t/p/w500/d8Ryb8AunYAuycV...
4,1160956,Panda Plan,International action star Jackie Chan is invit...,"[Action, Comedy]","[Jackie Chan, Shi Ce, Wei Xiang, Han Yanbo, Da...",[Zhang Luan],"[assistant, zoo, panda, superstar, rescue oper...",http://image.tmdb.org/t/p/w500/xVS9XiO9upp2SnW...


In [17]:
def remove_space(obj):
    name = []
    for i in obj:
        name.append(i.replace(" ",""))
    return " ".join(name)

In [18]:
# Text preprocessing
punctuations = string.punctuation
stop_words = stopwords.words('english')
ps = PorterStemmer()

def text_preprocessing(text):
    text = text.lower()
    text = word_tokenize(text)
    new_text = []
    
    for word in text:
        if word not in stop_words and word not in punctuations:
            new_text.append(word)
            
    y = []
    for word in new_text:
        y.append(ps.stem(word))
    return " ".join(y)

In [19]:
df['overview'] = df['overview'].apply(text_preprocessing)

In [20]:
df['genre'] = df['genre'].apply(remove_space)
df['cast'] = df['cast'].apply(remove_space)
df['crew'] = df['crew'].apply(remove_space)
df['keywords'] = df['keywords'].apply(remove_space)

In [21]:
df['tags'] = df['overview'] + df['genre'] + df['cast'] + df['crew'] + df['keywords']

In [22]:
df.head()

,id,title,overview,genre,cast,crew,keywords,image_url,tags
0,1241982,Moana 2,receiv unexpect call wayfind ancestor moana jo...,Animation Adventure Family Comedy,AuliʻiCravalho DwayneJohnson HualālaiChung Ros...,DavidG.DerrickJr. JasonHand DanaLedouxMiller,sea ocean villain musical sequel duringcredits...,http://image.tmdb.org/t/p/w500/aLVkiINlIeCkcZI...,receiv unexpect call wayfind ancestor moana jo...
1,927342,Amaran,heroic true stori major mukund varadarajan ind...,Action Drama Adventure War,Sivakarthikeyan SaiPallavi RahulBose BhuvanAro...,RajkumarPeriasamy,basedonnovelorbook affectation basedontruestor...,http://image.tmdb.org/t/p/w500/6m435uh40N7Gzfb...,heroic true stori major mukund varadarajan ind...
2,822119,Captain America: Brave New World,meet newli elect u.s. presid thaddeu ross sam ...,Action Thriller ScienceFiction,AnthonyMackie HarrisonFord DannyRamirez ShiraH...,JuliusOnah,hero superhero revenge blunt aftercreditssting...,http://image.tmdb.org/t/p/w500/pzIddUEMWhWzfvL...,meet newli elect u.s. presid thaddeu ross sam ...
3,939243,Sonic the Hedgehog 3,sonic knuckl tail reunit power new adversari s...,Action ScienceFiction Comedy Family,JimCarrey BenSchwartz KeanuReeves IdrisElba Co...,JeffFowler,moon sequel basedonvideogame burlesque dualrol...,http://image.tmdb.org/t/p/w500/d8Ryb8AunYAuycV...,sonic knuckl tail reunit power new adversari s...
4,1160956,Panda Plan,intern action star jacki chan invit adopt cere...,Action Comedy,JackieChan ShiCe WeiXiang HanYanbo DannyRay,ZhangLuan,assistant zoo panda superstar rescueoperation ...,http://image.tmdb.org/t/p/w500/xVS9XiO9upp2SnW...,intern action star jacki chan invit adopt cere...


In [23]:
df2 = df.copy()
df2 = df2[['id','title','tags','image_url']]

In [24]:
df2

,id,title,tags,image_url
0,1241982,Moana 2,receiv unexpect call wayfind ancestor moana jo...,http://image.tmdb.org/t/p/w500/aLVkiINlIeCkcZI...
1,927342,Amaran,heroic true stori major mukund varadarajan ind...,http://image.tmdb.org/t/p/w500/6m435uh40N7Gzfb...
2,822119,Captain America: Brave New World,meet newli elect u.s. presid thaddeu ross sam ...,http://image.tmdb.org/t/p/w500/pzIddUEMWhWzfvL...
3,939243,Sonic the Hedgehog 3,sonic knuckl tail reunit power new adversari s...,http://image.tmdb.org/t/p/w500/d8Ryb8AunYAuycV...
4,1160956,Panda Plan,intern action star jacki chan invit adopt cere...,http://image.tmdb.org/t/p/w500/xVS9XiO9upp2SnW...
...,...,...,...,...
14802,3111,A Star Is Born,movi star help young singer-actress find fame ...,http://image.tmdb.org/t/p/w500/zpg2SzpYhZk1D1s...
14803,239562,The Single Moms Club,group singl mom brought togeth aftermath incid...,http://image.tmdb.org/t/p/w500/cKVwaF5RzibSYbK...
14804,38579,Marmaduke,phil debbi winslow reloc nativ kansa sunni cli...,http://image.tmdb.org/t/p/w500/e1hxniKMiRIpGia...
14805,9989,Antitrust,comput programm 's dream job hot portland-bas ...,http://image.tmdb.org/t/p/w500/9PXyfcJe53A9qFk...


In [25]:
df2.reset_index(inplace = True)

# Text Vectorization

In [106]:
cv = CountVectorizer(ngram_range = (1,3), max_features = 5000)
tfidf = TfidfVectorizer(ngram_range = (1,3), max_features = 5000)

tags_cv = cv.fit_transform(df2['tags']).toarray()
tags_tfidf = tfidf.fit_transform(df2['tags']).toarray()

In [107]:
tags_cv.shape, tags_tfidf.shape

((8911, 5000), (8911, 5000))

In [108]:
similarity_cv = cosine_similarity(tags_cv)
similarity_tfidf = cosine_similarity(tags_tfidf)

In [109]:
def recommend_cv(name):
    movie_index = df2[df2['title'] == name].index[0]
    recommend_movie = sorted(enumerate(similarity_cv[movie_index]),key = (lambda x: x[1]), reverse = True)[:6]
    for i in recommend_movie:
        print(df2['title'].iloc[i[0]])
        print(df2['image_url'].iloc[i[0]])
        
def recommend_tfidf(name):
    movie_index = df2[df2['title'] == name].index[0]
    recommend_movie = sorted(enumerate(similarity_cv[movie_index]),key = (lambda x: x[1]), reverse = True)[:6]
    for i in recommend_movie:
        print(df2['title'].iloc[i[0]])
        print(df2['image_url'].iloc[i[0]])
        

In [110]:
recommend_cv('Harry Potter and the Philosopher\'s Stone')

Harry Potter and the Philosopher's Stone
http://image.tmdb.org/t/p/w500/wuMc08IPKEatf9rnMNXvIDxqP4W.jpg
Harry Potter and the Deathly Hallows: Part 2
http://image.tmdb.org/t/p/w500/c54HpQmuwXjHq2C9wmoACjxoom3.jpg
Harry Potter and the Prisoner of Azkaban
http://image.tmdb.org/t/p/w500/aWxwnYoe8p2d2fcxOqtvAtJ72Rw.jpg
Harry Potter and the Goblet of Fire
http://image.tmdb.org/t/p/w500/fECBtHlr0RB3foNHDiCBXeg9Bv9.jpg
Harry Potter and the Deathly Hallows: Part 1
http://image.tmdb.org/t/p/w500/iGoXIpQb7Pot00EEdwpwPajheZ5.jpg
Harry Potter and the Half-Blood Prince
http://image.tmdb.org/t/p/w500/z7uo9zmQdQwU5ZJHFpv2Upl30i1.jpg


In [111]:
recommend_tfidf('Harry Potter and the Philosopher\'s Stone')

Harry Potter and the Philosopher's Stone
http://image.tmdb.org/t/p/w500/wuMc08IPKEatf9rnMNXvIDxqP4W.jpg
Harry Potter and the Deathly Hallows: Part 2
http://image.tmdb.org/t/p/w500/c54HpQmuwXjHq2C9wmoACjxoom3.jpg
Harry Potter and the Prisoner of Azkaban
http://image.tmdb.org/t/p/w500/aWxwnYoe8p2d2fcxOqtvAtJ72Rw.jpg
Harry Potter and the Goblet of Fire
http://image.tmdb.org/t/p/w500/fECBtHlr0RB3foNHDiCBXeg9Bv9.jpg
Harry Potter and the Deathly Hallows: Part 1
http://image.tmdb.org/t/p/w500/iGoXIpQb7Pot00EEdwpwPajheZ5.jpg
Harry Potter and the Half-Blood Prince
http://image.tmdb.org/t/p/w500/z7uo9zmQdQwU5ZJHFpv2Upl30i1.jpg


In [105]:
recommend_cv('Matilda')

Matilda
http://image.tmdb.org/t/p/w500/wYoDpWInsBEVSmWStnRH06ddoyk.jpg
Harry Potter and the Philosopher's Stone
http://image.tmdb.org/t/p/w500/wuMc08IPKEatf9rnMNXvIDxqP4W.jpg
Lesson Plan
http://image.tmdb.org/t/p/w500/wawP3mOUeRBrAtnbPwWK5eFaMdV.jpg
Easy A
http://image.tmdb.org/t/p/w500/v5f1qO6NJnxWgONgkYg21TW39DT.jpg
Mean Girls 2
http://image.tmdb.org/t/p/w500/34zKHT98ZgSCz6ag1ZNBul7f1Wu.jpg
Sugar Baby
http://image.tmdb.org/t/p/w500/g9MaTZCsGGXjqF87I8XiFHp8Kby.jpg


In [51]:
df2.iloc[222]

index                                                      297
id                                                         424
title                                         Schindler's List
tags         true stori businessman oskar schindler save th...
image_url    http://image.tmdb.org/t/p/w500/sF1U4EUQS8YHUYj...
Name: 222, dtype: object

In [62]:
recommend_cv('Schindler\'s List')

Schindler's List
http://image.tmdb.org/t/p/w500/sF1U4EUQS8YHUYjNl3pMGNIQyr0.jpg
The Pianist
http://image.tmdb.org/t/p/w500/2hFvxCCWrTmCYwfy7yum0GKRi3Y.jpg
The Photographer of Mauthausen
http://image.tmdb.org/t/p/w500/6b4L0uLVgFsDfqPx0sFZbCRwIKH.jpg
Defiance
http://image.tmdb.org/t/p/w500/z6yNHcQOveRuAohuejdDFQDEBSs.jpg
The Zookeeper's Wife
http://image.tmdb.org/t/p/w500/5TVBtMhwOldZW82g1Ka11IE86wC.jpg
Lee
http://image.tmdb.org/t/p/w500/ffw51hmecHHdTWihncJZgmp9XH2.jpg


In [112]:
# df2.drop(columns = 'index', inplace = True)
import pickle
pickle.dump(similarity_cv,open('similarity.pkl','wb'))
# pickle.dump(df2,open('Movies Dataset.pkl','wb'))

In [97]:
df2[df2['title'] == 'Inkheart']

,id,title,tags,image_url
6937,2309,Inkheart,adventur father young daughter search long los...,http://image.tmdb.org/t/p/w500/pNTJVwXZZzXv5C2...


In [98]:
image[image['id'] == 6937]

,id,image_url
